In [13]:
## 알고리즘 문제, 장애물이 큐에 들어가는지 등
## 고정 장애물 만나도 rescan

## detectability를 정규화할 수 있는 방법?
## topography & cost 1~10으로?

## 장애물이 갑자기 사라지는 현상?
## 범위 안에 들어왔을 때 무조건 잡는게 너무 빡센듯

##장애물은 goal 못가게 함
##가만히 있으면 rescan 하도록 추가
##도착지 topography =0

In [58]:
import heapq
import sys
import numpy as np
import random
import pygame
import math
from itertools import combinations_with_replacement

In [59]:
#'xnym' -> [n,m]
def stateNameToCoords(name):
    return [int(name.split('x')[1].split('y')[0]), int(name.split('x')[1].split('y')[1])]

'''왜 sort가 주석 처리 되어있는지?'''
def topKey(queue):
    queue.sort()
    # print(queue)
    if len(queue) > 0:
        return queue[0][:2] #queue가장 위에 있는 것의 k1,k2만 가져옴
    else:
        #print('empty queue!')
        return (float('inf'), float('inf'))

def heuristic_from_s(Graph, id, s):
    id_coords=stateNameToCoords(id)
    s_coords=stateNameToCoords(s)
    
    x_distance =  (int(id_coords[0]) - int(s_coords[0]))**2 ##abs(int(id_coords[0]) - int(s_coords[0]))
    y_distance = (int(id_coords[1]) - int(s_coords[1]))**2 ##abs(int(id_coords[1]) - int(s_coords[1]))
    return np.sqrt(x_distance + y_distance)  ##x_distance + y_distance

def calculateKey(Graph, id, s_current, k_m):
    id_coords=stateNameToCoords(id)
    return (min(Graph.graph[id].g, Graph.graph[id].rhs) + heuristic_from_s(Graph, id, s_current) + k_m 
            + alpha * Graph.cells[id_coords[1]][id_coords[0]]
            + beta * Graph.topography[id_coords[1]][id_coords[0]], 
            min(Graph.graph[id].g, Graph.graph[id].rhs)
            + alpha * Graph.cells[id_coords[1]][id_coords[0]]
            + beta * Graph.topography[id_coords[1]][id_coords[0]]) 


def updateVertex(Graph, queue, id, s_current, k_m):
    #   s_goal = Graph.goal
    if id != Graph.goal:
        min_rhs = float('inf')
        for i in Graph.graph[id].children:
            min_rhs = min(min_rhs, Graph.graph[i].g + Graph.graph[id].children[i])
        Graph.graph[id].rhs = min_rhs
        
    id_in_queue = [item for item in queue if id in item]
    if id_in_queue != []:
        if len(id_in_queue) != 1:
            raise ValueError('more than one ' + id + ' in the queue!')
        queue.remove(id_in_queue[0])
    if Graph.graph[id].rhs != Graph.graph[id].g:
        '''장애물 위치도 queue에 들어가는지는 queue print해서 봐보자'''
        heapq.heappush(queue, calculateKey(Graph, id, s_current, k_m) + (id,))


def computeShortestPath(Graph, queue, s_start, k_m):
    while (Graph.graph[s_start].rhs != Graph.graph[s_start].g) or (topKey(queue) < calculateKey(Graph, s_start, s_start, k_m)):
        k_old = topKey(queue)
        u = heapq.heappop(queue)[2]
        
        if k_old < calculateKey(Graph, u, s_start, k_m):
            heapq.heappush(queue, calculateKey(Graph, u, s_start, k_m) + (u,))
        elif Graph.graph[u].g > Graph.graph[u].rhs:
            Graph.graph[u].g = Graph.graph[u].rhs
            for i in Graph.graph[u].parents:
                updateVertex(Graph, queue, i, s_start, k_m)
        else:
            Graph.graph[u].g = float('inf')
            updateVertex(Graph, queue, u, s_start, k_m)
            for i in Graph.graph[u].parents:
                updateVertex(Graph, queue, i, s_start, k_m)

def nextInShortestPath(Graph, s_current):
    min_rhs = float('inf')
    s_next = None
    if Graph.graph[s_current].rhs == float('inf'):
    #if Graph.graph[s_current].g == float('inf'):
        #print('You are done stuck')
        return s_current
    else:
        for i in Graph.graph[s_current].children:
            # print(i)
            child_cost = Graph.graph[i].g + Graph.graph[s_current].children[i]
            # print(child_cost)
            if (child_cost) < min_rhs:
                min_rhs = child_cost
                s_next = i
        if s_next:
            return s_next
        else:
            #raise ValueError('could not find child for transition!')
            #print('could not find child for transition!')
            return s_current

def scanForObstacles(Graph, queue, s_current, k_m):
    states_to_update = {}
    
    for neighbor in Graph.graph[s_current].children:
        neighbor_coords = stateNameToCoords(neighbor)
        states_to_update[neighbor] = Graph.cells[neighbor_coords[1]][neighbor_coords[0]]
        
    new_obstacle = False
    
    for state in states_to_update:
        if states_to_update[state] < 0:  # found cell with obstacle
            # print('found obstacle in ', state)
            for neighbor in Graph.graph[state].children:
                # first time to observe this obstacle where one wasn't before
                if(Graph.graph[state].children[neighbor] != float('inf')):
                    neighbor_coords = stateNameToCoords(state)
                    Graph.cells[neighbor_coords[1]][neighbor_coords[0]] = -2
                    Graph.graph[neighbor].children[state] = float('inf')
                    Graph.graph[state].children[neighbor] = float('inf')
                    updateVertex(Graph, queue, state, s_current, k_m)
                    new_obstacle = True
    return new_obstacle



def Move(Graph, s_current):
    if(s_current == Graph.goal):
        return 'goal'
    else:
        s_new = nextInShortestPath(Graph, s_current)
        new_coords = stateNameToCoords(s_new)
        if(Graph.cells[new_coords[1]][new_coords[0]] == -1):  # just ran into new obstacle
            s_new = s_current  # need to hold tight and scan/replan first
        
        return s_new
        
def Rescan(Graph, queue, s_current,k_m):
    results = scanForObstacles(Graph, queue, s_current, k_m)
    k_m += heuristic_from_s(Graph, s_current, Graph.goal)
    #s_last = s_current
    computeShortestPath(Graph, queue, s_current, k_m)

    return k_m
'''이전 moveAndRescan하고 의미가 달라지나?'''

#----------------------------------------------------------------------------

def initDStarLite(Graph, queue, s_start, s_goal, k_m):
    Graph.graph[s_goal].rhs = 0
    heapq.heappush(queue, calculateKey(Graph, s_goal, s_start, k_m) + (s_goal,))
    #print(queue[0])
    computeShortestPath(Graph, queue, s_start, k_m)
    return (Graph, queue, k_m)

In [60]:
def L1(a,b):
    ret=np.abs(a[0]-b[0])+np.abs(a[1]-b[1])
    if ret==0:
        return 0.1
    else:
        return ret

def L2(a,b):
    ret=np.sqrt(np.power((a[0]-b[0]),2)+np.power((a[1]-b[1]),2))
    if ret==0:
        return 0.1
    else:
        return ret

def softmax(X):
    exp_a = np.exp(X)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def get_move(move_prob):
    sample=np.random.rand()
    standard=move_prob[0]
    
    for i in range(len(move_prob)-1):
        if sample<standard:
            return i
        else:
            standard+=move_prob[i+1]
            
    return len(move_prob)-1

def move_obstacle(graph,pos_coords,move_chance=0.5,smart=False,detectability=1):
    if graph.obstacles:
        temp_obstacles=graph.obstacles.copy()
        for obstacle in graph.obstacles:
            if obstacle[2]:
                if random.random()<move_chance:
                    if smart:
                        move_prob=softmax([detectability*1/L2((pos_coords[1],pos_coords[0]),(obstacle[0][0]+i[0],obstacle[0][1]+i[1])) for i in move])
                        ## L1(대각선 이동 가능시 L2로 바꾸기) ## softmax로 확률 만들기 또는 그냥 합으로 나누기
                        dy,dx=move[get_move(move_prob)]
                            
                    else:
                        dy,dx=random.choice(move)
                        
                    new_x,new_y=obstacle[0][1]+dx,obstacle[0][0]+dy
                    if 0<=new_x<graph.x_dim and 0<=new_y<graph.y_dim and (new_y,new_x) not in [obs[0] for obs in temp_obstacles]:
                        if [new_x,new_y]!=stateNameToCoords(graph.goal):
                            temp_obstacles.remove(obstacle)
                            temp_obstacles.append(((new_y,new_x),obstacle[1],obstacle[2]))
                        
        graph.obstacles=temp_obstacles
        graph.setObstacle()

In [61]:
class Node:
    def __init__(self, id):
        self.id = id
        # dictionary of parent node ID's
        # key = id of parent
        # value = (edge cost,)
        self.parents = {}
        # dictionary of children node ID's
        # key = id of child
        # value = (edge cost,)
        self.children = {}
        # g approximation
        self.g = float('inf')
        # rhs value
        self.rhs = float('inf')

    def __str__(self):
        return 'Node: ' + self.id + ' g: ' + str(self.g) + ' rhs: ' + str(self.rhs)

    def __repr__(self):
        return self.__str__()

    def update_parents(self, parents):
        self.parents = parents


class Graph:
    def __init__(self):
        self.graph = {}

    def __str__(self):
        msg = 'Graph:'
        for i in self.graph:
            msg += '\n  node: ' + i + ' g: ' + str(self.graph[i].g) + ' rhs: ' + str(self.graph[i].rhs)
        return msg

    def __repr__(self):
        return self.__str__()

    def setStart(self, id):
        if(self.graph[id]):
            self.start = id
        else:
            raise ValueError('start id not in graph')

    def setGoal(self, id):
        if(self.graph[id]):
            self.goal = id
        else:
            raise ValueError('goal id not in graph')

In [62]:
#그래프 객체(장애물위치, 장애물cost, 지형지물 정보, 노드 정보 등)
class GridWorld(Graph):
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.obstacles=[] #((y,x),cost)
        self.cells=np.zeros((y_dim,x_dim))
        #self.topography=np.random.randint(1,11, size=(y_dim, x_dim))
        #self.topography=np.random.rand(y_dim,x_dim)
        self.topography=[[random.choice(range(1, 10)) / 10 for _ in range(x_dim)] for _ in range(y_dim)]
        self.topography[-1][-1]=0
        self.graph = {}
        self.generateGraphFromGrid()
        # self.printGrid()

    def __str__(self):
        msg = 'Graph:'
        for i in self.graph:
            msg += '\n  node: ' + i + ' g: ' + str(self.graph[i].g) + ' rhs: ' + str(self.graph[i].rhs) + ' neighbors: ' + str(self.graph[i].children)
        return msg

    def __repr__(self):
        return self.__str__()

    def printGrid(self):
        print('** GridWorld **')
        for row in self.cells:
            print(row)

    def printGValues(self):
        for j in range(self.y_dim):
            str_msg = ""
            for i in range(self.x_dim):
                node_id = 'x' + str(i) + 'y' + str(j)
                node = self.graph[node_id]
                if node.g == float('inf'):
                    str_msg += ' - '
                else:
                    str_msg += ' ' + str(node.g) + ' '
            print(str_msg)

    def generateGraphFromGrid(self):
        edge = 1
        cross=np.sqrt(2)
        for i in range(len(self.cells)):
            row = self.cells[i]
            for j in range(len(row)):
                # print('graph node ' + str(i) + ',' + str(j))
                node = Node('x' + str(j) + 'y' + str(i))
                if j > 0:  # not top row
                    node.parents['x' + str(j - 1) + 'y' + str(i)] = edge
                    node.children['x' + str(j - 1) + 'y' + str(i)] = edge
                if j + 1 < self.x_dim:  # not bottom row
                    node.parents['x' + str(j + 1) + 'y' + str(i)] = edge
                    node.children['x' + str(j + 1) + 'y' + str(i)] = edge
                if i > 0:  # not left col
                    node.parents['x' + str(j) + 'y' + str(i - 1)] = edge
                    node.children['x' + str(j) + 'y' + str(i - 1)] = edge
                if i + 1 < self.y_dim:  # not right col
                    node.parents['x' + str(j) + 'y' + str(i + 1)] = edge
                    node.children['x' + str(j) + 'y' + str(i + 1)] = edge
                if i > 0 and j > 0:
                    node.parents['x' + str(j - 1) + 'y' + str(i - 1)] = cross
                    node.children['x' + str(j - 1) + 'y' + str(i - 1)] = cross    
                if j > 0 and i + 1 < self.y_dim:
                    node.parents['x' + str(j - 1) + 'y' + str(i + 1)] = cross
                    node.children['x' + str(j - 1) + 'y' + str(i + 1)] = cross
                if j + 1 < self.x_dim and i > 0:
                    node.parents['x' + str(j + 1) + 'y' + str(i - 1)] = cross
                    node.children['x' + str(j + 1) + 'y' + str(i - 1)] = cross
                if j + 1 < self.x_dim and i + 1 < self.y_dim:
                    node.parents['x' + str(j + 1) + 'y' + str(i + 1)] = cross
                    node.children['x' + str(j + 1) + 'y' + str(i + 1)] = cross
                self.graph['x' + str(j) + 'y' + str(i)] = node
    
    #장애물 초기화 및 이동 반영
    def setObstacle(self):
        self.cells=np.zeros((self.y_dim,self.x_dim))
        for obs in self.obstacles:
            already=[]
            loc=obs[0]
            cost=obs[1]
                
            for i,c in enumerate(cost,start=1):
                if i==1:
                    candidate=move
                else:
                    candidate=[]
                    for a,*b in combinations_with_replacement(move, i):
                        candidate.append(tuple(sum(elem) for elem in zip(a, *b)))
                        
                    candidate=list(set(candidate)-set(already)-set([(0, 0)]))
                    
                already.extend(candidate)
                
                for m in candidate:
                    boundary=(loc[0]+m[0],loc[1]+m[1]) ##i배 할게 아니라 주변 다 삥 둘러서 cost줘야해

                    if [boundary[1],boundary[0]]==stateNameToCoords(self.goal):
                        continue

                    if (0<=boundary[0]<self.y_dim) and (0<=boundary[1]<self.x_dim): 
                        self.cells[boundary[0]][boundary[1]]+=c

        for obs in self.obstacles:
            loc=obs[0]
            self.cells[loc[0]][loc[1]]=-1

In [63]:
def initObstacles(Graph,obstacle_ratio,fixed_obs_ratio,influence=1):
    #obstacle grid크기에서 일정 비율만큼 랜덤 위치 생성 / obstacle cost 각각 랜덤 생성(0~1)
    max_obstacle_num=int(obstacle_ratio*Graph.x_dim*Graph.y_dim)
    fixed_obstacle_num=int(max_obstacle_num*fixed_obs_ratio)
    ob_num=0
    fix_num=0
    while ob_num<max_obstacle_num:
        ob_x=random.randint(0,Graph.x_dim-1)
        ob_y=random.randint(0,Graph.y_dim-1)
        
        if (ob_y,ob_x) in [obs[0] for obs in Graph.obstacles]:
            continue
        elif [ob_x,ob_y]==stateNameToCoords(Graph.goal):
            continue
        elif [ob_x,ob_y]==stateNameToCoords(Graph.start):
            continue
            
        if fix_num<fixed_obstacle_num:
            movable=0
            cost=0
            fix_num+=1
        else:
            movable=1
            #cost=random.randint(1,10)
            #cost=random.random()
            cost=random.choice(range(1, 10)) / 10
        
        cost_set=[]
        for i in range(1,influence+1):
            cost_set.append(cost/i)
        
        #print(((ob_y,ob_x),tuple(cost_set),movable))
        Graph.obstacles.append(((ob_y,ob_x),tuple(cost_set),movable))
        ob_num+=1
    
    Graph.setObstacle()

# Visualize

In [34]:
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED   = (255, 0, 0)
GRAY = (145, 145, 102)
DARKGRAY= (100, 100, 100)
BLUE = (0, 0, 200)
PINK = (255,0,255)

#장애물 및 현재위치 이동 가능한 경로
move=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
#move=[(-1,0),(0,-1),(0,1),(1,0)]

def colors(grid):
    if grid==0:
        return WHITE
    if grid==-1:
        return RED
    if grid==-2:
        return BLACK
    #return max((255-int((255-255)*grid/2),255-int((255-204)*grid/2),255-int((255-102)*grid/2)),(255,204,102))
    return max((255-int((255-255)*grid),255-int((255-204)*grid),255-int((255-102)*grid)),(255,204,102))

# This sets the width and height of each grid location
width = 30
height = 30

# This sets the margin between each cell
margin = 2

dimension_x =15
dimension_y =15
VIEWING_RANGE = 1

# obstacle num ratio of grid
obstacle_ratio=0.01
fixed_obs_ratio=0.9
move_chance=0.9
alpha = 1 #obstacle cost
beta = 1 #topography
smart=False
step=1
influence=2
detectability=1

# Set the height and width of the screen
WINDOW_SIZE = [(width + margin) * dimension_x + margin,
               (height + margin) * dimension_y + margin]

def draw_arrow(screen, colour, start, end):
    pygame.draw.line(screen,colour,start,end,1)
    rotation = math.degrees(math.atan2(start[1]-end[1], end[0]-start[0]))+90
    rad=12
    pygame.draw.polygon(screen, BLACK, ((end[0]+rad*math.sin(math.radians(rotation)), end[1]+rad*math.cos(math.radians(rotation))), (end[0]+rad*math.sin(math.radians(rotation-120)), end[1]+rad*math.cos(math.radians(rotation-120))), (end[0]+rad*math.sin(math.radians(rotation+120)), end[1]+rad*math.cos(math.radians(rotation+120)))))


transparent = pygame.Surface((3*width+3*margin,3*height+3*margin))  # the size of your rect
transparent.set_alpha(128)                # alpha level
transparent.fill(DARKGRAY)           # this fills the entire surface

<rect(0, 0, 96, 96)>

In [35]:
def main():
    
    # Loop until the user clicks the close button.
    done = False
    
    s_start=input("Enter start (xnym, where n,m are coordinates) : ")
    s_goal= input("Enter goal (xnym, where n,m are coordinates)  : ")
    #s_start = 'x1y2'
    #s_goal = 'x9y7'

    graph = GridWorld(dimension_x, dimension_y)
    
    goal_coords = stateNameToCoords(s_goal)
    start_coords= stateNameToCoords(s_start)

    graph.setStart(s_start)
    graph.setGoal(s_goal)
          
    k_m = 0
    s_last = s_start
    queue = []

    graph, queue, k_m = initDStarLite(graph, queue, s_start, s_goal, k_m)
    
    initObstacles(graph,obstacle_ratio,fixed_obs_ratio,influence)

    s_current = s_start
    pos_coords = stateNameToCoords(s_current)
    
    robot_centers=[]

    # Initialize pygame
    pygame.init()

    screen = pygame.display.set_mode(WINDOW_SIZE)

    # Set title of screen
    pygame.display.set_caption("D* Lite Path Planning")

    # Used to manage how fast the screen updates
    clock = pygame.time.Clock()
    basicfont = pygame.font.SysFont('Comic Sans MS', 22)

    # -------- Main Program Loop -----------
    while done==False:
        for event in pygame.event.get():   # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True  # Flag that we are done so we exit this loop
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                
                #현재위치가 obstacle한테 잡혔는지 확인
                if s_current in ['x{}y{}'.format(i[0][1],i[0][0]) for i in graph.obstacles]:
                    print('Failed !')
                    done=True
                    break
                
                #아군 먼저 이동
                print('move')
                s_new = Move(graph, s_current)

                if s_new == 'goal':
                    print('Goal Reached!')
                    done = True
                    break
                    
                pos_coords = stateNameToCoords(s_new)
                move_obstacle(graph,pos_coords,move_chance=move_chance,smart=smart,detectability=detectability)
                
                if s_current == s_new:
                    k_m = Rescan(graph, queue, s_current, k_m)
                else:
                    s_current = s_new

                    if step:
                        for obstacle in [obs for obs in graph.obstacles]:

                            loc=obstacle[0]
                            movable=obstacle[2]
                            gap_y = np.abs(pos_coords[1]-loc[0])
                            gap_x = np.abs(pos_coords[0]-loc[1])
                            if movable==0:
                                if gap_y<=1 and gap_x<=1:
                                    k_m = Rescan(graph, queue, s_current, k_m)
                                    break
                            elif movable==1:
                                if gap_y<=step and gap_x<=step:
                                    k_m = Rescan(graph, queue, s_current, k_m)
                                    break

                    else:
                        k_m = Rescan(graph, queue, s_current, k_m)

            

        # Set the screen background
        screen.fill(BLACK)


        # Draw the grid
        temp_grid=graph.cells.copy()
        temp_grid+=graph.topography#+temp_grid
        temp_grid=np.where(temp_grid<0,-1,temp_grid)
        
                
        for row in range(dimension_y):
            for column in range(dimension_x):
                
                COLOR =colors(temp_grid[row][column])
                if COLOR == RED:
                    if [i[2] for i in graph.obstacles if i[0]==(row,column)][0]==0:
                        COLOR = GRAY
                        
                pygame.draw.rect(screen, COLOR,
                                 [(margin + width) * column + margin,
                                  (margin + height) * row + margin, width, height])
                node_name = 'x' + str(column) + 'y' + str(row)


        # fill in goal cell with GREEN
        pygame.draw.rect(screen, GREEN, [(margin + width) * start_coords[0] + margin,
                                         (margin + height) * start_coords[1] + margin, width, height])


        # fill in goal cell with RED
        pygame.draw.rect(screen, BLUE, [(margin + width) * goal_coords[0] + margin,
                                         (margin + height) * goal_coords[1] + margin, width, height])

        # draw moving robot, based on pos_coords

        # Set the new robot centre
        robot_center = [int(pos_coords[0] * (width + margin) + width / 2) + margin , int(pos_coords[1] * (height + margin) + height / 2) + margin]
        draw_arrow(screen, BLACK, robot_center,robot_center)

        # maintain a list of all cells traversed
        robot_centers.append(robot_center)

        if len(robot_centers)>1:
            for i in range(0,len(robot_centers)-1):
                pygame.draw.line(screen,BLACK,robot_centers[i],robot_centers[i+1],3)

        # grey out visible boxes for robot
        screen.blit(transparent,(robot_center[0]-1.5*width-margin,robot_center[1]-1.5*height-margin))    # (0,0) are the top-left coordinates

        # Limit to 60 frames per second
        clock.tick(20)

        # Go ahead and update the screen with what we've drawn.
        pygame.display.flip()
        

    # Be IDLE friendly. If you forget this line, the program will 'hang'
    # on exit.
    pygame.quit()



if __name__ == "__main__":
    main()

Enter start (xnym, where n,m are coordinates) : x0y0
Enter goal (xnym, where n,m are coordinates)  : x14y14
move


2023-06-29 10:38:46.796 python[23171:1438083] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


move
move
move
move
move
move
move
move
move
move
move
move
move
move
Goal Reached!


# Repeat Test

In [64]:
import heapq
from tqdm import tqdm

move=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
#move=[(-1,0),(0,-1),(0,1),(1,0)]

def stateNameToCoords(name):
    return [int(name.split('x')[1].split('y')[0]), int(name.split('x')[1].split('y')[1])]

In [65]:
def iter_test(test_num, dimension_x, dimension_y, s_start, s_goal,
              obstacle_ratio, fixed_obs_ratio, move_chance, 
              step, smart, influence, detectability):

    success = 0
    move_num_list = []
    succ_time_list = []
    import time
    for idx in tqdm(range(test_num)):
        start=time.time()
        # Loop until the user clicks the close button.
        done = False

        graph = GridWorld(dimension_x, dimension_y)

        graph.setStart(s_start)
        graph.setGoal(s_goal)

        k_m = 0
        s_last = s_start
        queue = []

        graph, queue, k_m = initDStarLite(graph, queue, s_start, s_goal, k_m)

        initObstacles(graph,obstacle_ratio,fixed_obs_ratio,influence)

        s_current = s_start
        pos_coords = stateNameToCoords(s_current)

        # -------- Main Program Loop -----------
        move_num=0
        while done==False:

            if s_current in ['x{}y{}'.format(i[0][1],i[0][0]) for i in graph.obstacles]:
                #print(f'{idx}. Failed !')
                done=True
                break

            s_new = Move(graph, s_current)
            move_num+=1
            if s_new == 'goal':
                succ_time=time.time()-start
                done = True
                #print(f'{idx}. Success ! --> move {move_num} times')
                move_num_list.append(move_num)
                succ_time_list.append(succ_time)
                success+=1
                break
            else:
                pos_coords = stateNameToCoords(s_new)
                move_obstacle(graph,pos_coords,move_chance=move_chance,smart=smart,detectability=detectability)

                if s_current == s_new:
                    k_m = Rescan(graph, queue, s_current, k_m)

                else :
                    s_current = s_new
                    if step:
                        for obstacle in [obs for obs in graph.obstacles]:

                            loc=obstacle[0]
                            movable=obstacle[2]
                            gap_y = np.abs(pos_coords[1]-loc[0])
                            gap_x = np.abs(pos_coords[0]-loc[1])
                            if movable==0:
                                if gap_y<=1 and gap_x<=1:
                                    k_m = Rescan(graph, queue, s_current, k_m)
                                    break
                            elif movable==1:
                                if gap_y<=step and gap_x<=step:
                                    k_m = Rescan(graph, queue, s_current, k_m)
                                    break

                    else:
                        k_m = Rescan(graph, queue, s_current, k_m)
                
    print('\n-Success Ratio : {}%'.format(100*success/test_num))
    print('-Average moving times : {}'.format(np.mean(move_num_list)))
    return 100*success/test_num, np.mean(move_num_list), np.mean(succ_time_list)

#### graph size : Grid 크기
#### obstacle_ratio : Grid 크기에서 모든 장애물 비율
#### fixed_obs_ratio : 모든 장애물에서 고정 장애물 비율
#### move_chance : 가변 장애물의 매번 움직일 확률
#### alpha : calculate_key에서 k1,k2에 더해주는 가변 장애물 주변 cost의 가중치
#### beta : calculate_key에서 k1,k2에 더해주는 지형지물의 가중치
#### step : 나의 위치가 가변 장애물의 몇칸안에 들어왔을 때 Rescan할지
#### influence : 가변 장애물 cost의 영역 크기
#### smart : 가변 장애물이 나의 위치를 기반하에 움직일지 랜덤하게 움직일지
#### detectability : smart=True일 때, 나의 위치 감지력
--------------------------------------------------------------------------

#### graph size : 10x10, 15x15, 20x20
#### obstacle_ratio : 0.05, 0.1, 0.15, 0.2
#### fixed_obs_ratio : 0.6, 0.7, 0.8, 0.9
#### move_chance : 0.3, 0.5, 0.7, 1.0
#### alpha : 0, 1, 5, 10, 15, 20, 50
#### beta : 0, 1, 3, 5, 7, 10
#### step : 1, 2
#### influence : 1, 2, 3
#### smart : False, True
#### detectability : 1, 5, 10, 30 --smart 일때만 유효

In [66]:
import itertools
import pickle

# 변수와 값의 조합 생성
variables = ['graph_size', 'obstacle_ratio', 'fixed_obstacle','move_chance','influence','detectability']
values = [[15,30,45],
          [0.01,0.03],
          [0.9],
          [0.5,0.9],
          [1,2],
         [5,10]]
combinations = list(itertools.product(*values))

# 테스트 실행
for idx,combination in enumerate(combinations,start=1):
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence,detectability = combination
    print(idx,graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability)

1 15 0.01 0.9 0.5 1 5
2 15 0.01 0.9 0.5 1 10
3 15 0.01 0.9 0.5 2 5
4 15 0.01 0.9 0.5 2 10
5 15 0.01 0.9 0.9 1 5
6 15 0.01 0.9 0.9 1 10
7 15 0.01 0.9 0.9 2 5
8 15 0.01 0.9 0.9 2 10
9 15 0.03 0.9 0.5 1 5
10 15 0.03 0.9 0.5 1 10
11 15 0.03 0.9 0.5 2 5
12 15 0.03 0.9 0.5 2 10
13 15 0.03 0.9 0.9 1 5
14 15 0.03 0.9 0.9 1 10
15 15 0.03 0.9 0.9 2 5
16 15 0.03 0.9 0.9 2 10
17 30 0.01 0.9 0.5 1 5
18 30 0.01 0.9 0.5 1 10
19 30 0.01 0.9 0.5 2 5
20 30 0.01 0.9 0.5 2 10
21 30 0.01 0.9 0.9 1 5
22 30 0.01 0.9 0.9 1 10
23 30 0.01 0.9 0.9 2 5
24 30 0.01 0.9 0.9 2 10
25 30 0.03 0.9 0.5 1 5
26 30 0.03 0.9 0.5 1 10
27 30 0.03 0.9 0.5 2 5
28 30 0.03 0.9 0.5 2 10
29 30 0.03 0.9 0.9 1 5
30 30 0.03 0.9 0.9 1 10
31 30 0.03 0.9 0.9 2 5
32 30 0.03 0.9 0.9 2 10
33 45 0.01 0.9 0.5 1 5
34 45 0.01 0.9 0.5 1 10
35 45 0.01 0.9 0.5 2 5
36 45 0.01 0.9 0.5 2 10
37 45 0.01 0.9 0.9 1 5
38 45 0.01 0.9 0.9 1 10
39 45 0.01 0.9 0.9 2 5
40 45 0.01 0.9 0.9 2 10
41 45 0.03 0.9 0.5 1 5
42 45 0.03 0.9 0.5 1 10
43 45 0.03 0.9 0.5 2 5

## result

In [66]:
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_False.pkl','rb') as f:
    result_dict1F=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_False.pkl','rb') as f:
    result_dict2F=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True.pkl','rb') as f:
    result_dict1T=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_True.pkl','rb') as f:
    result_dict2T=pickle.load(f)

In [11]:
ab11='alpha: 1, beta: 1'
ab110='alpha: 1, beta: 10'
ab120='alpha: 1, beta: 20'
ab101='alpha: 10, beta: 1'
ab1010='alpha: 10, beta: 10'
ab1020='alpha: 10, beta: 20'
ab201='alpha: 20, beta: 1'
ab2010='alpha: 20, beta: 10'
ab2020='alpha: 20, beta: 20'

In [21]:
ab11_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab11 in key]
ab110_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab110 in key]
ab120_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab120 in key]
ab101_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab101 in key]
ab1010_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab1010 in key]
ab1020_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab1020 in key]
ab201_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab201 in key]
ab2010_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab2010 in key]
ab2020_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab2020 in key]

In [22]:
ab11_mean=np.mean(ab11_ret,axis=0)
ab110_mean=np.mean(ab110_ret,axis=0)
ab120_mean=np.mean(ab120_ret,axis=0)
ab101_mean=np.mean(ab101_ret,axis=0)
ab1010_mean=np.mean(ab1010_ret,axis=0)
ab1020_mean=np.mean(ab1020_ret,axis=0)
ab201_mean=np.mean(ab201_ret,axis=0)
ab2010_mean=np.mean(ab2010_ret,axis=0)
ab2020_mean=np.mean(ab2020_ret,axis=0)

In [23]:
print('alpha:1 / beta:1 ->',ab11_mean)
print('alpha:1 / beta:10 ->',ab110_mean)
print('alpha:1 / beta:20 ->',ab120_mean)
print('alpha:10 / beta:1 ->',ab101_mean)
print('alpha:10 / beta:10 ->',ab1010_mean)
print('alpha:10 / beta:20 ->',ab1020_mean)
print('alpha:20 / beta:1 ->',ab201_mean)
print('alpha:20 / beta:10 ->',ab2010_mean)
print('alpha:20 / beta:20 ->',ab2020_mean)

alpha:1 / beta:1 -> [55.69791667 25.18852934 83.50106442]
alpha:1 / beta:10 -> [ 53.6875      27.31624583 149.64226622]
alpha:1 / beta:20 -> [ 51.53125     28.74757116 304.63843212]
alpha:10 / beta:1 -> [ 55.16666667  25.46314735 103.42585159]
alpha:10 / beta:10 -> [ 53.84375     27.85826464 172.79618169]
alpha:10 / beta:20 -> [ 52.61458333  29.23686262 461.41389106]
alpha:20 / beta:1 -> [ 56.15625     24.99809343 119.43259797]
alpha:20 / beta:10 -> [ 54.48958333  26.92968507 194.38173715]
alpha:20 / beta:20 -> [ 53.01041667  29.70717448 593.09795533]


- step1 smart False

    alpha:20 / beta:10 -> [ 93.3125      42.5476714  187.90822463]

    alpha:20 / beta:1 -> [ 93.09375     32.91195981 112.78751773]
  
            
- step2 smart False

    alpha:10 / beta:1 -> [ 93.36458333  30.54942061 104.91763587]
    
    alpha:20 / beta:1 -> [ 93.08333333  31.0260814  118.83881807]
            
- step1 smart True

    alpha:10 / beta:1 -> [ 55.96875     25.41193349 100.68952364]
    
    alpha:20 / beta:1 -> [ 55.54166667  24.82433499 115.27375411]
            
- step2 smart True

    alpha:20 / beta:1 -> [ 56.15625     24.99809343 119.43259797]
    
    alpha:1 / beta:1 -> [55.69791667 25.18852934 83.50106442]

In [29]:
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_False_second.pkl','rb') as f:
    result_dict1F_s=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_False_second.pkl','rb') as f:
    result_dict2F_s=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_second.pkl','rb') as f:
    result_dict1T_s=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_True_second.pkl','rb') as f:
    result_dict2T_s=pickle.load(f)

In [68]:
ab505='alpha: 5, beta: 0.5'
ab51='alpha: 5, beta: 1'
ab53='alpha: 5, beta: 3'
ab1005='alpha: 10, beta: 0.5'
ab101='alpha: 10, beta: 1'
ab103='alpha: 10, beta: 3'
ab1505='alpha: 15, beta: 0.5'
ab151='alpha: 15, beta: 1'
ab153='alpha: 15, beta: 3'
ab2005='alpha: 20, beta: 0.5'
ab201='alpha: 20, beta: 1'
ab203='alpha: 20, beta: 3'
ab2505='alpha: 25, beta: 0.5'
ab251='alpha: 25, beta: 1'
ab253='alpha: 25, beta: 3'
ab155='alpha: 15, beta: 5'
ab1510='alpha: 15, beta: 10'
ab1515='alpha: 15, beta: 15'
ab205='alpha: 20, beta: 5'
ab2010='alpha: 20, beta: 10'
ab2015='alpha: 20, beta: 15'
ab255='alpha: 25, beta: 5'
ab2510='alpha: 25, beta: 10'
ab2515='alpha: 25, beta: 15'

In [82]:
ab1505_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab1505 in key]
ab151_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab151 in key]
ab153_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab153 in key]
ab2005_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab2005 in key]
ab201_ret=[result_dict2T[key] for key in list(result_dict2T.keys()) if ab201 in key]
ab203_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab203 in key]
ab2505_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab2505 in key]
ab251_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab251 in key]
ab253_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab253 in key]

In [83]:
ab505_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab505 in key]
ab51_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab51 in key]
ab53_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab53 in key]
ab1005_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab1005 in key]
ab101_ret=[result_dict1T[key] for key in list(result_dict1T.keys()) if ab101 in key]
ab103_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab103 in key]
ab1505_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab1505 in key]
ab151_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab151 in key]
ab153_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab153 in key]

In [91]:
ab505_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab505 in key]
ab51_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab51 in key]
ab53_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab53 in key]
ab1005_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab1005 in key]
ab101_ret=[result_dict2F[key] for key in list(result_dict2F.keys()) if ab101 in key]
ab103_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab103 in key]
ab1505_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab1505 in key]
ab151_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab151 in key]
ab153_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab153 in key]

In [94]:
ab155_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab155 in key]
ab1510_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab1510 in key]
ab1515_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab1515 in key]
ab205_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab205 in key]
ab2010_ret=[result_dict1F[key] for key in list(result_dict1F.keys()) if ab2010 in key]
ab2015_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab2015 in key]
ab255_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab255 in key]
ab2510_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab2510 in key]
ab2515_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab2515 in key]

In [84]:
ab1505_mean=np.mean(ab1505_ret,axis=0)
ab151_mean=np.mean(ab151_ret,axis=0)
ab153_mean=np.mean(ab153_ret,axis=0)
ab2005_mean=np.mean(ab2005_ret,axis=0)
ab201_mean=np.mean(ab201_ret,axis=0)
ab203_mean=np.mean(ab203_ret,axis=0)
ab2505_mean=np.mean(ab2505_ret,axis=0)
ab251_mean=np.mean(ab251_ret,axis=0)
ab253_mean=np.mean(ab253_ret,axis=0)

In [92]:
ab505_mean=np.mean(ab505_ret,axis=0)
ab51_mean=np.mean(ab51_ret,axis=0)
ab53_mean=np.mean(ab53_ret,axis=0)
ab1005_mean=np.mean(ab1005_ret,axis=0)
ab101_mean=np.mean(ab101_ret,axis=0)
ab103_mean=np.mean(ab103_ret,axis=0)
ab1505_mean=np.mean(ab1505_ret,axis=0)
ab151_mean=np.mean(ab151_ret,axis=0)
ab153_mean=np.mean(ab153_ret,axis=0)

In [95]:
ab155_mean=np.mean(ab155_ret,axis=0)
ab1510_mean=np.mean(ab1510_ret,axis=0)
ab1515_mean=np.mean(ab1515_ret,axis=0)
ab205_mean=np.mean(ab205_ret,axis=0)
ab2010_mean=np.mean(ab2010_ret,axis=0)
ab2015_mean=np.mean(ab2015_ret,axis=0)
ab255_mean=np.mean(ab255_ret,axis=0)
ab2510_mean=np.mean(ab2510_ret,axis=0)
ab2515_mean=np.mean(ab2515_ret,axis=0)

In [86]:
print('alpha:15 / beta:0.5 ->',ab1505_mean)
print('alpha:15 / beta:1 ->',ab151_mean)
print('alpha:15 / beta:3 ->',ab153_mean)
print('alpha:20 / beta:0.5 ->',ab2005_mean)
print('alpha:20 / beta:1 ->',ab201_mean)
print('alpha:20 / beta:3 ->',ab203_mean)
print('alpha:25 / beta:0.5 ->',ab2505_mean)
print('alpha:25 / beta:1 ->',ab251_mean)
print('alpha:25 / beta:3 ->',ab253_mean)

alpha:15 / beta:0.5 -> [57.32291667 23.03930986 65.75345043]
alpha:15 / beta:1 -> [57.23958333 23.07607362 65.39834172]
alpha:15 / beta:3 -> [ 56.23958333  24.00916569 102.71284298]
alpha:20 / beta:0.5 -> [58.1875     23.05504613 76.1066101 ]
alpha:20 / beta:1 -> [ 56.15625     24.99809343 119.43259797]
alpha:20 / beta:3 -> [56.02083333 23.86336522 63.15269542]
alpha:25 / beta:0.5 -> [58.01041667 23.06733992 83.11998712]
alpha:25 / beta:1 -> [57.91666667 23.06748609 81.23762859]
alpha:25 / beta:3 -> [  55.66666667   24.04697571 1938.76476386]


In [93]:
print('alpha:5 / beta:0.5 ->',ab505_mean)
print('alpha:5 / beta:1 ->',ab51_mean)
print('alpha:5 / beta:3 ->',ab53_mean)
print('alpha:10 / beta:0.5 ->',ab1005_mean)
print('alpha:10 / beta:1 ->',ab101_mean)
print('alpha:10 / beta:3 ->',ab103_mean)
print('alpha:15 / beta:0.5 ->',ab1505_mean)
print('alpha:15 / beta:1 ->',ab151_mean)
print('alpha:15 / beta:3 ->',ab153_mean)

alpha:5 / beta:0.5 -> [93.04166667 23.135507   45.90002564]
alpha:5 / beta:1 -> [93.04166667 23.14154068 49.34668761]
alpha:5 / beta:3 -> [93.3125     25.36126046 82.30670449]
alpha:10 / beta:0.5 -> [93.33333333 23.14412825 58.28567201]
alpha:10 / beta:1 -> [ 93.36458333  30.54942061 104.91763587]
alpha:10 / beta:3 -> [92.60416667 24.5535692  96.19605485]
alpha:15 / beta:0.5 -> [93.16666667 23.12106928 65.18131039]
alpha:15 / beta:1 -> [93.60416667 23.15160909 68.8787196 ]
alpha:15 / beta:3 -> [ 93.625       25.9622846  106.76806708]


In [96]:
print('alpha:15 / beta:5 ->',ab155_mean)
print('alpha:15 / beta:10 ->',ab1510_mean)
print('alpha:15 / beta:15 ->',ab1515_mean)
print('alpha:20 / beta:5 ->',ab205_mean)
print('alpha:20 / beta:10 ->',ab2010_mean)
print('alpha:20 / beta:15 ->',ab2015_mean)
print('alpha:25 / beta:5 ->',ab255_mean)
print('alpha:25 / beta:10 ->',ab2510_mean)
print('alpha:25 / beta:15 ->',ab2515_mean)

alpha:15 / beta:5 -> [ 93.72916667  28.50552306 163.13862564]
alpha:15 / beta:10 -> [ 92.5         37.71170476 335.09045023]
alpha:15 / beta:15 -> [ 91.6875      51.3912614  487.84847537]
alpha:20 / beta:5 -> [ 93.95833333  32.36108245 174.33949456]
alpha:20 / beta:10 -> [ 93.3125      42.5476714  187.90822463]
alpha:20 / beta:15 -> [ 92.58333333  48.46915795 515.81873949]
alpha:25 / beta:5 -> [ 92.85416667  31.53927002 187.65516792]
alpha:25 / beta:10 -> [ 92.70833333  36.72805734 355.32334597]
alpha:25 / beta:15 -> [ 92.79166667  49.02349292 531.27826285]


- step1 smart False

    alpha:20 / beta:5 -> [ 93.95833333  32.36108245 174.33949456]
    
    alpha:15 / beta:5 -> [ 93.72916667  28.50552306 163.13862564]
    
            
- step2 smart False

    alpha:15 / beta:3 -> [ 93.625       25.9622846  106.76806708]
    
    alpha:15 / beta:1 -> [93.60416667 23.15160909 68.8787196 ]
    
            
- step1 smart True

    alpha:10 / beta:0.5 -> [57.91666667 23.04667896 54.6506304 ]
    
    alpha:15 / beta:0.5 -> [57.32291667 23.03930986 65.75345043]
    
    alpha:5 / beta:0.5 -> [57.32291667 23.02037719 42.8804839 ]
    
            
- step2 smart True

    alpha:20 / beta:0.5 -> [58.1875     23.05504613 76.1066101 ]
    
    alpha:25 / beta:0.5 -> [58.01041667 23.06733992 83.11998712]

In [ ]:
20 5 / 17 5 / 23 5 / 20 2 / 17 2 / 23 2 / 20 8 / 17 8 / 23 8
15 3 / 12 3 / 18 3 / 15 2 / 12 2 / 18 2 / 15 5 / 12 5 / 18 5 / 
10 0.5 / 7 0.5 / 13 0.5
20 0.5 / 17 0.5 / 23 0.5

In [25]:
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_False_third.pkl','rb') as f:
    result_dict1F_t=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_False_third.pkl','rb') as f:
    result_dict2F_t=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_third.pkl','rb') as f:
    result_dict1T_t=pickle.load(f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_True_third.pkl','rb') as f:
    result_dict2T_t=pickle.load(f)

In [39]:
ab205='alpha: 20, beta: 5'
ab175='alpha: 17, beta: 5'
ab235='alpha: 23, beta: 5'
ab202='alpha: 20, beta: 2'
ab172='alpha: 17, beta: 2'
ab232='alpha: 23, beta: 2'
ab208='alpha: 20, beta: 8'
ab178='alpha: 17, beta: 8'
ab238='alpha: 23, beta: 8'
ab153='alpha: 15, beta: 3'
ab123='alpha: 12, beta: 3'
ab183='alpha: 18, beta: 3'
ab152='alpha: 15, beta: 2'
ab122='alpha: 12, beta: 2'
ab182='alpha: 18, beta: 2'
ab155='alpha: 15, beta: 5'
ab125='alpha: 12, beta: 5'
ab185='alpha: 18, beta: 5'
ab1005='alpha: 10, beta: 0.5'
ab705='alpha: 7, beta: 0.5'
ab1305='alpha: 13, beta: 0.5'
ab2005='alpha: 20, beta: 0.5'
ab1705='alpha: 17, beta: 0.5'
ab2305='alpha: 23, beta: 0.5'

In [30]:
ab205_ret=[result_dict1F_s[key] for key in list(result_dict1F_s.keys()) if ab205 in key]
ab175_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab175 in key]
ab235_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab235 in key]
ab202_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab202 in key]
ab172_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab172 in key]
ab232_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab232 in key]
ab208_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab208 in key]
ab178_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab178 in key]
ab238_ret=[result_dict1F_t[key] for key in list(result_dict1F_t.keys()) if ab238 in key]

In [40]:
ab153_ret=[result_dict2F_s[key] for key in list(result_dict2F_s.keys()) if ab153 in key]
ab123_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab123 in key]
ab183_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab183 in key]
ab152_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab152 in key]
ab122_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab122 in key]
ab182_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab182 in key]
ab155_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab155 in key]
ab125_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab125 in key]
ab185_ret=[result_dict2F_t[key] for key in list(result_dict2F_t.keys()) if ab185 in key]

In [43]:
ab1005_ret=[result_dict1T_s[key] for key in list(result_dict1T_s.keys()) if ab1005 in key]
ab705_ret=[result_dict1T_t[key] for key in list(result_dict1T_t.keys()) if ab705 in key]
ab1305_ret=[result_dict1T_t[key] for key in list(result_dict1T_t.keys()) if ab1305 in key]

In [46]:
ab2005_ret=[result_dict2T_s[key] for key in list(result_dict2T_s.keys()) if ab2005 in key]
ab1705_ret=[result_dict2T_t[key] for key in list(result_dict2T_t.keys()) if ab1705 in key]
ab2305_ret=[result_dict2T_t[key] for key in list(result_dict2T_t.keys()) if ab2305 in key]

In [31]:
ab205_mean=np.mean(ab205_ret,axis=0)
ab175_mean=np.mean(ab175_ret,axis=0)
ab235_mean=np.mean(ab235_ret,axis=0)
ab202_mean=np.mean(ab202_ret,axis=0)
ab172_mean=np.mean(ab172_ret,axis=0)
ab232_mean=np.mean(ab232_ret,axis=0)
ab208_mean=np.mean(ab208_ret,axis=0)
ab178_mean=np.mean(ab178_ret,axis=0)
ab238_mean=np.mean(ab238_ret,axis=0)

In [41]:
ab153_mean=np.mean(ab153_ret,axis=0)
ab123_mean=np.mean(ab123_ret,axis=0)
ab183_mean=np.mean(ab183_ret,axis=0)
ab152_mean=np.mean(ab152_ret,axis=0)
ab122_mean=np.mean(ab122_ret,axis=0)
ab182_mean=np.mean(ab182_ret,axis=0)
ab155_mean=np.mean(ab155_ret,axis=0)
ab125_mean=np.mean(ab125_ret,axis=0)
ab185_mean=np.mean(ab185_ret,axis=0)

In [44]:
ab1005_mean=np.mean(ab1005_ret,axis=0)
ab705_mean=np.mean(ab705_ret,axis=0)
ab1305_mean=np.mean(ab1305_ret,axis=0)

In [47]:
ab2005_mean=np.mean(ab2005_ret,axis=0)
ab1705_mean=np.mean(ab1705_ret,axis=0)
ab2305_mean=np.mean(ab2305_ret,axis=0)

In [32]:
print('alpha:20 / beta:5 ->',ab205_mean)
print('alpha:17 / beta:5 ->',ab175_mean)
print('alpha:23 / beta:5 ->',ab235_mean)
print('alpha:20 / beta:2 ->',ab202_mean)
print('alpha:17 / beta:2 ->',ab172_mean)
print('alpha:23 / beta:2 ->',ab232_mean)
print('alpha:20 / beta:8 ->',ab208_mean)
print('alpha:17 / beta:8 ->',ab178_mean)
print('alpha:23 / beta:8 ->',ab238_mean)

alpha:20 / beta:5 -> [ 93.95833333  32.36108245 174.33949456]
alpha:17 / beta:5 -> [ 93.33333333  32.49321256 171.48074199]
alpha:23 / beta:5 -> [ 92.64583333  28.62608617 178.68276224]
alpha:20 / beta:2 -> [92.97916667 23.58843224 85.49845831]
alpha:17 / beta:2 -> [93.27083333 23.63424028 82.20425577]
alpha:23 / beta:2 -> [92.9375    23.6297846 87.9339297]
alpha:20 / beta:8 -> [ 93.54166667  38.13575554 274.91976249]
alpha:17 / beta:8 -> [ 93.625       33.86281182 270.21093704]
alpha:23 / beta:8 -> [ 92.77083333  37.7310265  280.40688646]


In [42]:
print('alpha:15 / beta:3 ->',ab153_mean)
print('alpha:12 / beta:3 ->',ab123_mean)
print('alpha:18 / beta:3 ->',ab183_mean)
print('alpha:15 / beta:2 ->',ab152_mean)
print('alpha:12 / beta:2 ->',ab122_mean)
print('alpha:18 / beta:2 ->',ab182_mean)
print('alpha:15 / beta:5 ->',ab155_mean)
print('alpha:12 / beta:5 ->',ab125_mean)
print('alpha:18 / beta:5 ->',ab185_mean)

alpha:15 / beta:3 -> [ 93.625       25.9622846  106.76806708]
alpha:12 / beta:3 -> [92.77083333 24.11764    99.02637926]
alpha:18 / beta:3 -> [ 93.4375      26.25675108 111.18076733]
alpha:15 / beta:2 -> [93.39583333 23.5141637  79.75961785]
alpha:12 / beta:2 -> [93.125      23.49913074 79.25447984]
alpha:18 / beta:2 -> [92.625      23.49360288 87.22319171]
alpha:15 / beta:5 -> [ 92.8125      29.73458757 170.49162087]
alpha:12 / beta:5 -> [ 92.52083333  26.46202523 164.13253923]
alpha:18 / beta:5 -> [ 93.625       32.24599914 176.73363686]


In [45]:
print('alpha:10 / beta:0.5 ->',ab1005_mean)
print('alpha:7 / beta:0.5 ->',ab705_mean)
print('alpha:13 / beta:0.5 ->',ab1305_mean)

alpha:10 / beta:0.5 -> [57.91666667 23.04667896 54.6506304 ]
alpha:7 / beta:0.5 -> [57.22916667 23.02641299 48.65415409]
alpha:13 / beta:0.5 -> [57.64583333 23.04232561 61.25731428]


In [48]:
print('alpha:20 / beta:0.5 ->',ab2005_mean)
print('alpha:17 / beta:0.5 ->',ab1705_mean)
print('alpha:23 / beta:0.5 ->',ab2305_mean)

alpha:20 / beta:0.5 -> [58.1875     23.05504613 76.1066101 ]
alpha:17 / beta:0.5 -> [57.54166667 23.03779689 72.47846473]
alpha:23 / beta:0.5 -> [58.17708333 23.05234682 82.06402921]


- step1 smart False

    alpha:20 / beta:5 -> [ 93.95833333  32.36108245 174.33949456]
    
    alpha:17 / beta:8 -> [ 93.625       33.86281182 270.21093704]
    
            
- step2 smart False

    alpha:15 / beta:3 -> [ 93.625       25.9622846  106.76806708]
    
    alpha:18 / beta:5 -> [ 93.625       32.24599914 176.73363686]
    
            
- step1 smart True

    alpha:10 / beta:0.5 -> [57.91666667 23.04667896 54.6506304 ]
    
    alpha:13 / beta:0.5 -> [57.64583333 23.04232561 61.25731428]
    
            
- step2 smart True

    alpha:20 / beta:0.5 -> [58.1875     23.05504613 76.1066101 ]
    
    alpha:23 / beta:0.5 -> [58.17708333 23.05234682 82.06402921]

### step=1 & smart=False

In [67]:
result_dict1={} # 1,True
result_dict2={} # 2,True
result_dict3={} # 1,False
result_dict4={} # 2,False

In [19]:
step=1
smart=False

In [20]:
alpha=20
beta=5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {1}'
    succ,mov_num,time=iter_test(test_num=100, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict3[pre_condition+', '+string]=[succ,mov_num,time]

# with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
#     pickle.dump(result_dict,f)
# print(pre_condition,'saved !')


 step: 1, smart: False, alpha: 20, beta: 5 


 0 start 



100%|█████████████████████████████████████████| 100/100 [00:10<00:00,  9.32it/s]



-Success Ratio : 92.0%
-Average moving times : 16.22826086956522

 1 start 



100%|█████████████████████████████████████████| 100/100 [00:10<00:00,  9.09it/s]



-Success Ratio : 96.0%
-Average moving times : 16.135416666666668

 2 start 



...


-Success Ratio : 95.0%
-Average moving times : 48.43157894736842

 22 start 



100%|█████████████████████████████████████████| 100/100 [05:02<00:00,  3.02s/it]



-Success Ratio : 91.0%
-Average moving times : 47.67032967032967

 23 start 



100%|█████████████████████████████████████████| 100/100 [05:35<00:00,  3.35s/it]


-Success Ratio : 92.0%
-Average moving times : 47.91304347826087


In [22]:
alpha=15
beta=3

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {1}'
    succ,mov_num,time=iter_test(test_num=100, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict4[pre_condition+', '+string]=[succ,mov_num,time]

# with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
#     pickle.dump(result_dict,f)
# print(pre_condition,'saved !')


 step: 2, smart: False, alpha: 15, beta: 3 


 0 start 



100%|█████████████████████████████████████████| 100/100 [00:05<00:00, 17.05it/s]



-Success Ratio : 97.0%
-Average moving times : 15.938144329896907

 1 start 



100%|█████████████████████████████████████████| 100/100 [00:07<00:00, 13.36it/s]



-Success Ratio : 98.0%
-Average moving times : 15.86734693877551

 2 start 



...


-Success Ratio : 92.0%
-Average moving times : 46.869565217391305

 22 start 



100%|█████████████████████████████████████████| 100/100 [03:17<00:00,  1.97s/it]



-Success Ratio : 91.0%
-Average moving times : 46.79120879120879

 23 start 



100%|█████████████████████████████████████████| 100/100 [03:51<00:00,  2.31s/it]


-Success Ratio : 86.0%
-Average moving times : 46.883720930232556


In [68]:
step=2
smart=True

In [69]:
alpha=100
beta=0.5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict2[pre_condition+', '+string]=[succ,mov_num,time]

# with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
#     pickle.dump(result_dict,f)
# print(pre_condition,'saved !')


 step: 2, smart: True, alpha: 100, beta: 0.5 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:14<00:00, 20.13it/s]



-Success Ratio : 77.33333333333333%
-Average moving times : 15.051724137931034

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:15<00:00, 19.14it/s]



-Success Ratio : 77.0%
-Average moving times : 15.077922077922079

 2 start 



...


-Success Ratio : 29.0%
-Average moving times : 46.03448275862069

 46 start 



100%|█████████████████████████████████████████| 300/300 [18:17<00:00,  3.66s/it]



-Success Ratio : 37.333333333333336%
-Average moving times : 46.19642857142857

 47 start 



100%|█████████████████████████████████████████| 300/300 [18:58<00:00,  3.80s/it]


-Success Ratio : 33.0%
-Average moving times : 46.04040404040404


In [70]:
step=1
smart=True

In [71]:
alpha=100
beta=0.5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict1[pre_condition+', '+string]=[succ,mov_num,time]

# with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
#     pickle.dump(result_dict,f)
# print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 100, beta: 0.5 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:14<00:00, 20.38it/s]



-Success Ratio : 79.0%
-Average moving times : 15.10548523206751

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:19<00:00, 15.58it/s]



-Success Ratio : 73.33333333333333%
-Average moving times : 15.072727272727272

 2 start 



...


-Success Ratio : 30.0%
-Average moving times : 45.94444444444444

 46 start 



100%|█████████████████████████████████████████| 300/300 [13:42<00:00,  2.74s/it]



-Success Ratio : 41.0%
-Average moving times : 46.08130081300813

 47 start 



100%|█████████████████████████████████████████| 300/300 [18:46<00:00,  3.75s/it]


-Success Ratio : 26.0%
-Average moving times : 45.98717948717949


In [72]:
with open('/Users/sewoong/Desktop/ML/미래도전/alpha_100_step1True','wb') as f:
    pickle.dump(result_dict1,f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/alpha_100_step2True','wb') as f:
    pickle.dump(result_dict2,f)

In [27]:
import pickle

with open('/Users/sewoong/Desktop/ML/미래도전/succ_avgtime_1T','wb') as f:
    pickle.dump(result_dict1,f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/succ_avgtime_2T','wb') as f:
    pickle.dump(result_dict2,f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/succ_avgtime_1F','wb') as f:
    pickle.dump(result_dict3,f)
    
with open('/Users/sewoong/Desktop/ML/미래도전/succ_avgtime_2F','wb') as f:
    pickle.dump(result_dict4,f)

In [30]:
import os

a = '/Users/sewoong/Desktop/ML'

os.path.join(a)

'/Users/sewoong/Desktop/ML'

In [49]:
succ_time = 0
count = 0
for key,item in result_dict1.items():
    if 'detectability: 10' in key:
        #print(item)
        count+=1
        succ_time+=item[2]
#     succ_time+=item[2]

for key,item in result_dict2.items():
    if 'detectability: 10' in key:
        #print(item)
        count+=1
        succ_time+=item[2]
#     succ_time+=item[2]
    
succ_time/(len(result_dict1)+len(result_dict2))
succ_time/count

0.38432971875828414

In [50]:
succ_time = 0
count = 0
for key,item in result_dict3.items():
    #print(item)
    succ_time+=item[2]

for key,item in result_dict4.items():
    #print(item)
    succ_time+=item[2]
    
succ_time/(len(result_dict3)+len(result_dict4))

0.985066751356102

In [57]:
succ_time = 0
count = 0
for key,item in result_dict1.items():
    if 'move_chance: 0.9' in key:
        count+=1
        succ_time+=item[2]
        
for key,item in result_dict2.items():
    if 'move_chance: 0.9' in key:
        count+=1
        succ_time+=item[2]
        
for key,item in result_dict3.items():
    if 'move_chance: 0.9' in key:
        count+=1
        succ_time+=item[2]

for key,item in result_dict4.items():
    if 'move_chance: 0.9' in key:
        count+=1
        succ_time+=item[2]
    
succ_time/count

0.4941766533828765

In [ ]:
succ_time = 0
count = 0
for key,item in result_dict1.items():
    if 'graph_size: 45' in key:
        count+=1
        succ_time+=item[2]
        
for key,item in result_dict2.items():
    if 'graph_size: 45' in key:
        count+=1
        succ_time+=item[2]
        
for key,item in result_dict3.items():
    if 'graph_size: 45' in key:
        count+=1
        succ_time+=item[2]

for key,item in result_dict4.items():
    if 'graph_size: 45' in key:
        count+=1
        succ_time+=item[2]
    
succ_time/count

In [ ]:
alpha=20
beta=0.5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=100, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict2[pre_condition+', '+string]=[succ,mov_num,time]

# with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
#     pickle.dump(result_dict,f)
# print(pre_condition,'saved !')

In [78]:
alpha=10
beta=0.5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_final.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 10, beta: 0.5 


 0 start 



100%|███████████████████████████████████████████| 300/300 [00:09<00:00, 32.41it/s]



-Success Ratio : 78.0%
-Average moving times : 15.047008547008547

 1 start 



100%|███████████████████████████████████████████| 300/300 [00:09<00:00, 31.05it/s]



-Success Ratio : 77.33333333333333%
-Average moving times : 15.077586206896552

 2 start 



...


-Success Ratio : 30.333333333333332%
-Average moving times : 45.989010989010985

 46 start 



100%|███████████████████████████████████████████| 300/300 [05:40<00:00,  1.14s/it]



-Success Ratio : 43.666666666666664%
-Average moving times : 46.06870229007634

 47 start 



100%|███████████████████████████████████████████| 300/300 [05:54<00:00,  1.18s/it]


-Success Ratio : 32.0%
-Average moving times : 46.020833333333336
step: 1, smart: True, alpha: 10, beta: 0.5 saved !


In [59]:
alpha=15
beta=3

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {1}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_False_final.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 2, smart: False, alpha: 15, beta: 3 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:21<00:00, 14.11it/s]



-Success Ratio : 95.66666666666667%
-Average moving times : 15.759581881533101

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:23<00:00, 12.91it/s]



-Success Ratio : 98.0%
-Average moving times : 15.82312925170068

 2 start 



...

100%|█████████████████████████████████████████| 300/300 [05:00<00:00,  1.00s/it]



-Success Ratio : 97.33333333333333%
-Average moving times : 46.20205479452055

 17 start 



100%|█████████████████████████████████████████| 300/300 [06:22<00:00,  1.27s/it]



-Success Ratio : 97.33333333333333%
-Average moving times : 46.1986301369863

 18 start 



 75%|██████████████████████████████▉          | 226/300 [04:12<01:07,  1.10it/s]

In [22]:
alpha=12
beta=5

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {1}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_2_smart_False_third.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 2, smart: False, alpha: 12, beta: 5 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:30<00:00,  9.69it/s]



-Success Ratio : 96.66666666666667%
-Average moving times : 19.989655172413794

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:32<00:00,  9.20it/s]



-Success Ratio : 96.66666666666667%
-Average moving times : 16.182758620689654

 2 start 



...


-Success Ratio : 89.0%
-Average moving times : 40.1310861423221

 14 start 



100%|█████████████████████████████████████████| 300/300 [05:23<00:00,  1.08s/it]



-Success Ratio : 84.33333333333333%
-Average moving times : 33.88142292490119

 15 start 



100%|█████████████████████████████████████████| 300/300 [06:32<00:00,  1.31s/it]


-Success Ratio : 84.33333333333333%
-Average moving times : 35.14229249011858
step: 2, smart: False, alpha: 12, beta: 5 saved !


In [135]:
alpha=20
beta=2

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {1}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_False_third.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: False, alpha: 20, beta: 2 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:17<00:00, 17.46it/s]



-Success Ratio : 96.33333333333333%
-Average moving times : 15.474048442906575

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:18<00:00, 15.82it/s]



-Success Ratio : 96.33333333333333%
-Average moving times : 15.498269896193772

 2 start 



...


-Success Ratio : 92.0%
-Average moving times : 31.681159420289855

 14 start 



100%|█████████████████████████████████████████| 300/300 [03:10<00:00,  1.57it/s]



-Success Ratio : 86.66666666666667%
-Average moving times : 31.78846153846154

 15 start 



100%|█████████████████████████████████████████| 300/300 [03:51<00:00,  1.30it/s]


-Success Ratio : 86.66666666666667%
-Average moving times : 31.807692307692307
step: 1, smart: False, alpha: 20, beta: 2 saved !


In [31]:
alpha=5
beta=1

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_second.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 5, beta: 1 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:08<00:00, 33.87it/s]



-Success Ratio : 75.33333333333333%
-Average moving times : 15.106194690265486

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:10<00:00, 29.17it/s]



-Success Ratio : 75.0%
-Average moving times : 15.097777777777777

 2 start 



...


-Success Ratio : 21.0%
-Average moving times : 31.206349206349206

 30 start 



100%|█████████████████████████████████████████| 300/300 [01:57<00:00,  2.55it/s]



-Success Ratio : 28.0%
-Average moving times : 31.44047619047619

 31 start 



100%|█████████████████████████████████████████| 300/300 [02:00<00:00,  2.48it/s]


-Success Ratio : 19.666666666666668%
-Average moving times : 31.28813559322034
step: 1, smart: True, alpha: 5, beta: 1 saved !


In [16]:
alpha=5
beta=3

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations[24:]):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_second.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 5, beta: 3 


 0 start 



100%|█████████████████████████████████████████| 300/300 [02:46<00:00,  1.80it/s]



-Success Ratio : 51.0%
-Average moving times : 40.48366013071895

 1 start 



100%|█████████████████████████████████████████| 300/300 [02:45<00:00,  1.81it/s]



-Success Ratio : 48.666666666666664%
-Average moving times : 32.56164383561644

 2 start 



...


-Success Ratio : 14.666666666666666%
-Average moving times : 33.88636363636363

 6 start 



100%|█████████████████████████████████████████| 300/300 [02:52<00:00,  1.73it/s]



-Success Ratio : 29.333333333333332%
-Average moving times : 31.829545454545453

 7 start 



100%|█████████████████████████████████████████| 300/300 [02:53<00:00,  1.72it/s]


-Success Ratio : 22.333333333333332%
-Average moving times : 32.134328358208954
step: 1, smart: True, alpha: 5, beta: 3 saved !


In [15]:
alpha=15
beta=1

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/미래도전/step_1_smart_True_second.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 15, beta: 1 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:12<00:00, 23.17it/s]



-Success Ratio : 72.0%
-Average moving times : 15.11111111111111

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:13<00:00, 22.89it/s]



-Success Ratio : 77.33333333333333%
-Average moving times : 15.112068965517242

 2 start 



...


-Success Ratio : 16.333333333333332%
-Average moving times : 31.020408163265305

 30 start 



100%|█████████████████████████████████████████| 300/300 [02:47<00:00,  1.79it/s]



-Success Ratio : 35.666666666666664%
-Average moving times : 31.261682242990656

 31 start 



100%|█████████████████████████████████████████| 300/300 [03:24<00:00,  1.46it/s]


-Success Ratio : 19.0%
-Average moving times : 31.280701754385966
step: 1, smart: True, alpha: 15, beta: 1 saved !


In [29]:
alpha=15
beta=3

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations[:11]): #11~
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence, detectability = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: {detectability}'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=detectability)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/step_1_smart_True_second.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: True, alpha: 15, beta: 3 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:21<00:00, 14.08it/s]



-Success Ratio : 74.33333333333333%
-Average moving times : 15.825112107623319

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:22<00:00, 13.57it/s]



-Success Ratio : 73.0%
-Average moving times : 15.771689497716896

 2 start 



...


-Success Ratio : 57.0%
-Average moving times : 16.34502923976608

 9 start 



100%|█████████████████████████████████████████| 300/300 [00:32<00:00,  9.22it/s]



-Success Ratio : 52.666666666666664%
-Average moving times : 16.18354430379747

 10 start 



100%|█████████████████████████████████████████| 300/300 [00:33<00:00,  9.03it/s]


-Success Ratio : 61.333333333333336%
-Average moving times : 16.184782608695652
step: 1, smart: True, alpha: 15, beta: 3 saved !


In [53]:
alpha=20
beta=10

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: 1'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/step_2_smart_False.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 2, smart: False, alpha: 20, beta: 10 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:28<00:00, 10.51it/s]



-Success Ratio : 95.66666666666667%
-Average moving times : 18.32752613240418

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:31<00:00,  9.66it/s]



-Success Ratio : 96.0%
-Average moving times : 17.305555555555557

 2 start 



...


-Success Ratio : 93.0%
-Average moving times : 47.08243727598566

 14 start 



100%|█████████████████████████████████████████| 300/300 [06:15<00:00,  1.25s/it]



-Success Ratio : 80.0%
-Average moving times : 42.52916666666667

 15 start 



100%|█████████████████████████████████████████| 300/300 [07:24<00:00,  1.48s/it]


-Success Ratio : 88.66666666666667%
-Average moving times : 34.03007518796993
step: 2, smart: False, alpha: 20, beta: 10 saved !


In [33]:
alpha=20
beta=20

pre_condition=f'step: {step}, smart: {smart}, alpha: {alpha}, beta: {beta}'
print('\n',pre_condition,'\n')

for idx,combination in enumerate(combinations[:-3]):
    print('\n',idx,'start','\n')
    graph_size, obstacle_ratio, fixed_obstacle, move_chance, influence = combination
    string=f'graph_size: {graph_size}, obstacle_ratio: {obstacle_ratio}, fixed_obs_ratio: {fixed_obstacle}, move_chance: {move_chance}, influence: {influence}, detectability: 1'
    succ,mov_num,time=iter_test(test_num=300, dimension_x=graph_size, dimension_y=graph_size, s_start='x0y0', s_goal=f'x{graph_size-1}y{graph_size-1}',
              obstacle_ratio=obstacle_ratio, fixed_obs_ratio=fixed_obstacle, move_chance=move_chance, 
              step=step, smart=smart, influence=influence, detectability=1)
    result_dict[pre_condition+', '+string]=[succ,mov_num,time]

with open('/Users/sewoong/Desktop/ML/step_1_smart_False.pkl','wb') as f:
    pickle.dump(result_dict,f)
print(pre_condition,'saved !')


 step: 1, smart: False, alpha: 20, beta: 20 


 0 start 



100%|█████████████████████████████████████████| 300/300 [00:46<00:00,  6.45it/s]



-Success Ratio : 95.33333333333333%
-Average moving times : 29.594405594405593

 1 start 



100%|█████████████████████████████████████████| 300/300 [00:48<00:00,  6.17it/s]



-Success Ratio : 93.66666666666667%
-Average moving times : 19.28825622775801

 2 start 



...


-Success Ratio : 97.0%
-Average moving times : 83.19587628865979

 11 start 



100%|█████████████████████████████████████████| 300/300 [09:25<00:00,  1.88s/it]



-Success Ratio : 97.0%
-Average moving times : 145.16494845360825

 12 start 



100%|█████████████████████████████████████████| 300/300 [13:23<00:00,  2.68s/it]


-Success Ratio : 92.0%
-Average moving times : 55.79710144927536
step: 1, smart: False, alpha: 20, beta: 20 saved !


In [ ]:
3,6 empty_queue, done stuck